<a href="https://colab.research.google.com/github/syedmehrabzaidi/AI-LAB/blob/main/fine_tune_medical_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📦 Cell 1: Install Required Libraries
Start by installing necessary libraries:

In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

##**Create a Tiny Sample Dataset**

prepeared data set of 300 to 400 records for entity extraction




In [ ]:
#meducal dataset


import random
import json
from datasets import Dataset

# Symptoms and related medications
symptoms_to_medications = {
    "fever": ["Paracetamol", "Ibuprofen", "Aspirin"],
    "headache": ["Paracetamol", "Ibuprofen", "Sumatriptan"],
    "cold": ["Cetirizine", "Loratadine", "Levocetirizine"],
    "cough": ["Dextromethorphan", "Codeine", "Guaifenesin"],
    "diarrhea": ["Loperamide", "ORS", "Racecadotril"],
    "vomiting": ["Ondansetron", "Domperidone", "Metoclopramide"],
    "motion sickness": ["Dimenhydrinate", "Meclizine", "Promethazine"],
    "body pain": ["Paracetamol", "Naproxen", "Diclofenac"],
    "allergy": ["Cetirizine", "Chlorpheniramine", "Fexofenadine"],
    "acidity": ["Pantoprazole", "Omeprazole", "Ranitidine"]
}

# Natural language templates
templates = [
    "I have been having {symptom}. What medicine should I take?",
    "Can you suggest tablets for {symptom}?",
    "What are the best medications for treating {symptom}?",
    "I'm suffering from {symptom}, any recommended drugs?",
    "What tablets help with {symptom}?",
    "Please suggest some medicine for {symptom}.",
    "I'm experiencing {symptom}. What can I take?",
]

def generate_medical_sample():
    symptom = random.choice(list(symptoms_to_medications.keys()))
    meds = random.sample(symptoms_to_medications[symptom], k=random.randint(1, len(symptoms_to_medications[symptom])))
    prompt = random.choice(templates).format(symptom=symptom)
    response_text = f"For {symptom}, you can take " + ", ".join(meds[:-1]) + (" and " if len(meds) > 1 else "") + meds[-1] + "."

    # Build entity list
    entities = [{"text": med, "type": "Medication"} for med in meds]

    return {
        "prompt": prompt,
        "response": json.dumps({"entities": entities})
    }

# Generate 420 examples
synthetic_data = [generate_medical_sample() for _ in range(420)]
print(synthetic_data[:5])  # View sample

# Convert to HuggingFace dataset
dataset = Dataset.from_list(synthetic_data)
dataset


[{'prompt': "I'm suffering from acidity, any recommended drugs?", 'response': '{"entities": [{"text": "Pantoprazole", "type": "Medication"}, {"text": "Ranitidine", "type": "Medication"}]}'}, {'prompt': 'Can you suggest tablets for body pain?', 'response': '{"entities": [{"text": "Paracetamol", "type": "Medication"}, {"text": "Naproxen", "type": "Medication"}, {"text": "Diclofenac", "type": "Medication"}]}'}, {'prompt': "I'm suffering from fever, any recommended drugs?", 'response': '{"entities": [{"text": "Ibuprofen", "type": "Medication"}, {"text": "Paracetamol", "type": "Medication"}]}'}, {'prompt': 'I have been having body pain. What medicine should I take?', 'response': '{"entities": [{"text": "Naproxen", "type": "Medication"}]}'}, {'prompt': "I'm suffering from headache, any recommended drugs?", 'response': '{"entities": [{"text": "Paracetamol", "type": "Medication"}]}'}]


Dataset({
    features: ['prompt', 'response'],
    num_rows: 420
})

In [ ]:
"""
import random
import json
from datasets import Dataset

# Define entity pools
names = ["Alice Johnson", "Bob Smith", "Charlie Davis", "Diana Green", "Ethan Brown", "Fiona White", "George Young", "Hannah Black", "Isaac Moore", "Julia Scott"]
companies = ["Google", "OpenAI", "Microsoft", "Apple", "Meta", "Amazon", "Netflix", "Tesla", "Nvidia", "Adobe"]
professions = ["engineer", "designer", "manager", "researcher", "scientist", "consultant", "analyst", "developer", "specialist", "director"]
locations = ["New York", "San Francisco", "London", "Berlin", "Toronto", "Tokyo", "Paris", "Sydney", "Singapore", "Dubai"]
weather = ["sunny", "rainy", "snowy", "cloudy", "stormy", "foggy", "windy", "humid"]
cars = ["Tesla Model 3", "Ford Mustang", "Chevrolet Camaro", "BMW 3 Series", "Audi A4", "Mercedes Benz C-Class", "Honda Civic", "Toyota Corolla", "Porsche 911", "Nissan Altima"]
events = ["conference", "festival", "webinar", "convention", "hackathon", "workshop", "trade show", "meeting"]
products = ["iPhone 14", "Galaxy S23", "MacBook Pro", "Surface Laptop", "PlayStation 5", "Nintendo Switch", "iPad", "Kindle Paperwhite", "GoPro Hero 10"]

# Define templates for more variability
templates = {
    "person": [
        "{name} works at {company} as a {profession}.",
        "As a {profession}, {name} joined {company} last year.",
        "{company} hired {name} for their expertise as a {profession}."
    ],
    "company": [
        "{company} launched a new product today.",
        "{company} announced a major acquisition this morning.",
        "{company} is expanding its operations globally."
    ],
    "weather": [
        "The weather today is {weather_condition} in {location}.",
        "{location} is experiencing a {weather_condition} day.",
        "It's been quite {weather_condition} recently in {location}."
    ],
    "car": [
        "The new {car_model} is available in {location}.",
        "I saw a {car_model} driving through {location} yesterday.",
        "Auto dealers in {location} now offer the {car_model}."
    ],
    "event": [
        "The {event} will take place in {location} next week.",
        "{location} will host the annual {event} this month.",
        "A global {event} is scheduled soon in {location}."
    ],
    "product": [
        "The new {product} was released by {company}.",
        "{company} unveiled the {product} in a live event.",
        "Tech fans are excited about the {product} launched by {company}."
    ]
}

def generate_sample():
    name = random.choice(names)
    company = random.choice(companies)
    profession = random.choice(professions)
    location = random.choice(locations)
    weather_condition = random.choice(weather)
    car_model = random.choice(cars)
    event = random.choice(events)
    product = random.choice(products)

    entity_type = random.choice(list(templates.keys()))
    template = random.choice(templates[entity_type])

    sentence = template.format(
        name=name,
        company=company,
        profession=profession,
        location=location,
        weather_condition=weather_condition,
        car_model=car_model,
        event=event,
        product=product
    )

    if entity_type == "person":
        entities = [{"text": name, "type": "Person"}, {"text": company, "type": "Organization"}]
    elif entity_type == "company":
        entities = [{"text": company, "type": "Organization"}]
    elif entity_type == "weather":
        entities = [{"text": weather_condition, "type": "Weather"}, {"text": location, "type": "Location"}]
    elif entity_type == "car":
        entities = [{"text": car_model, "type": "Car"}, {"text": location, "type": "Location"}]
    elif entity_type == "event":
        entities = [{"text": event, "type": "Event"}, {"text": location, "type": "Location"}]
    elif entity_type == "product":
        entities = [{"text": product, "type": "Product"}, {"text": company, "type": "Organization"}]

    return {
        "prompt": f"Extract entities in JSON format from: {sentence}",
        "response": json.dumps({"entities": entities})
    }

# Generate dataset
synthetic_data = [generate_sample() for _ in range(1200)]  # Increase for more variety

# Preview sample
print(synthetic_data[:3])

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(synthetic_data)
print(dataset)
"""


'\nimport random\nimport json\nfrom datasets import Dataset\n\n# Define entity pools\nnames = ["Alice Johnson", "Bob Smith", "Charlie Davis", "Diana Green", "Ethan Brown", "Fiona White", "George Young", "Hannah Black", "Isaac Moore", "Julia Scott"]\ncompanies = ["Google", "OpenAI", "Microsoft", "Apple", "Meta", "Amazon", "Netflix", "Tesla", "Nvidia", "Adobe"]\nprofessions = ["engineer", "designer", "manager", "researcher", "scientist", "consultant", "analyst", "developer", "specialist", "director"]\nlocations = ["New York", "San Francisco", "London", "Berlin", "Toronto", "Tokyo", "Paris", "Sydney", "Singapore", "Dubai"]\nweather = ["sunny", "rainy", "snowy", "cloudy", "stormy", "foggy", "windy", "humid"]\ncars = ["Tesla Model 3", "Ford Mustang", "Chevrolet Camaro", "BMW 3 Series", "Audi A4", "Mercedes Benz C-Class", "Honda Civic", "Toyota Corolla", "Porsche 911", "Nissan Altima"]\nevents = ["conference", "festival", "webinar", "convention", "hackathon", "workshop", "trade show", "meeti

In [ ]:
"""


import random
import json
from datasets import Dataset

# Sample data to randomize
names = ["Alice Johnson", "Bob Smith", "Charlie Davis", "Diana Green", "Ethan Brown", "Fiona White", "George Young", "Hannah Black", "Isaac Moore", "Julia Scott"]
companies = ["Google", "OpenAI", "Microsoft", "Apple", "Facebook", "Amazon", "Netflix", "Tesla", "Nvidia", "Adobe"]
professions = ["engineer", "designer", "manager", "researcher", "scientist", "consultant", "analyst", "developer", "specialist", "director"]
locations = ["New York", "San Francisco", "London", "Berlin", "Toronto", "Tokyo", "Paris", "Sydney", "Singapore", "Dubai"]

def generate_sample():
    name = random.choice(names)
    company = random.choice(companies)
    profession = random.choice(professions)
    location = random.choice(locations)

    templates = [
        f"{name} works at {company} as a {profession}.",
        f"{name} is a {profession} at {company} based in {location}.",
        f"{company} employs {name} as a {profession}.",
        f"In {location}, {name} works as a {profession} at {company}.",
        f"{name}, a {profession}, recently joined {company} in {location}."
    ]

    sentence = random.choice(templates)

    # Build response JSON
    entities = [
        {"text": name, "type": "Person"},
        {"text": company, "type": "Organization"},
        {"text": location, "type": "Location"} if "in " + location in sentence or "based in " + location in sentence else None
    ]
    entities = [e for e in entities if e is not None]

    return {
        "prompt": f"Extract entities in JSON format from: {sentence}",
        "response": json.dumps({"entities": entities})
    }

# Generate 400+ samples
synthetic_data = [generate_sample() for _ in range(420)]
print(synthetic_data)
# Convert to HuggingFace Dataset
dataset = Dataset.from_list(synthetic_data)
dataset
"""


'\n\n\nimport random\nimport json\nfrom datasets import Dataset\n\n# Sample data to randomize\nnames = ["Alice Johnson", "Bob Smith", "Charlie Davis", "Diana Green", "Ethan Brown", "Fiona White", "George Young", "Hannah Black", "Isaac Moore", "Julia Scott"]\ncompanies = ["Google", "OpenAI", "Microsoft", "Apple", "Facebook", "Amazon", "Netflix", "Tesla", "Nvidia", "Adobe"]\nprofessions = ["engineer", "designer", "manager", "researcher", "scientist", "consultant", "analyst", "developer", "specialist", "director"]\nlocations = ["New York", "San Francisco", "London", "Berlin", "Toronto", "Tokyo", "Paris", "Sydney", "Singapore", "Dubai"]\n\ndef generate_sample():\n    name = random.choice(names)\n    company = random.choice(companies)\n    profession = random.choice(professions)\n    location = random.choice(locations)\n\n    templates = [\n        f"{name} works at {company} as a {profession}.",\n        f"{name} is a {profession} at {company} based in {location}.",\n        f"{company} em

In [ ]:
from transformers import AutoTokenizer

model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Falcon doesn't have a pad token

def preprocess(example):
    full_prompt = f"{example['prompt']}\n{example['response']}"
    tokenized = tokenizer(full_prompt, truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

## Loading the model raw without fine tuning to check results before finetuning





In [ ]:
from transformers import AutoModelForCausalLM
import torch

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
base_model.resize_token_embeddings(len(tokenizer))



config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Embedding(50257, 2048)

## Generate response

In [ ]:
def extract_entities_raw(text):
    prompt = f"Extract entities in JSON format from: {text}"
    inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
    with torch.no_grad():
        outputs = base_model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(extract_entities_raw("Elon Musk founded Tesla and SpaceX."))


Extract entities in JSON format from: Elon Musk founded Tesla and SpaceX. He is also the CEO of SolarCity, a solar energy company. He is also the founder of OpenAI, a company that develops artificial intelligence.
The company is based in California.
The company is based in California.
The company is based in California.
The company is based in California.
The company is based in California.
The company is based in California.
The company is based in California.
The company is based in California.
The company is based in


## ⚙️ Set Training Configuration

We use PEFT (LoRA) + TRL's SFTTrainer to train efficiently even on a T4 GPU.


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

from peft import prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

base_model.enable_input_require_grads()
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()



trainable params: 1,572,864 || all params: 1,313,101,824 || trainable%: 0.1198


## train the model and save finetuned weights

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="falcon-entity-extractor-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    num_train_epochs=3,
    fp16=True,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()
model.save_pretrained("falcon-entity-extractor-lora")
tokenizer.save_pretrained("falcon-entity-extractor-lora")



<ipython-input-9-8978f8251423>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## load weights for evaluation

In [ ]:
from peft import PeftModel

from transformers import AutoModelForCausalLM
model_ft = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model_ft.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(model_ft, "falcon-entity-extractor-lora")
model.eval()



In [ ]:
def extract_entities(text, max_new_tokens=100):
    prompt = f"Extract entities in JSON format from: {text}"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "\n" in decoded:
        return decoded.split("\n", 1)[-1].strip()
    return decoded.strip()

# ✅ Test After Fine-Tuning
print(extract_entities("what tablets should i take for my headacs"))
